In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [3]:
input_shape = (128, 128, 3) #RGB_photos

In [4]:
# Define the CNN architecture for feature extraction using the Functional API
inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3))(inputs)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

In [5]:
# Create the feature extractor model with defined input and output
feature_extractor = Model(inputs=inputs, outputs=x)

In [6]:
# Compile the feature extractor model
feature_extractor.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# Create an instance of ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [8]:
# Set the directory paths for the training and test images
train_dir = 'C:/Users/DELL/Downloads/New_folder/SEC_Four/train'
test_dir = 'C:/Users/DELL/Downloads/New_folder/SEC_Four/test'

In [9]:
# Set the batch size and number of epochs
batch_size = 32
epochs = 10

In [10]:
# Load and preprocess the training images using ImageDataGenerator
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Load and preprocess the test images using ImageDataGenerator
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 4010 images belonging to 2 classes.
Found 3543 images belonging to 2 classes.


In [11]:
# Extract features for each image in the training set
train_features = feature_extractor.predict(train_generator)
train_labels = train_generator.classes  # Get the labels for training data

C:\Users\DELL\anaconda3\New folder\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step


In [12]:
# Extract features for each image in the test set
test_features = feature_extractor.predict(test_generator)
test_labels = test_generator.classes  # Get the labels for test data

 35/111 ━━━━━━━━━━━━━━━━━━━━ 9s 124ms/step

C:\Users\DELL\anaconda3\New folder\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


111/111 ━━━━━━━━━━━━━━━━━━━━ 23s 205ms/step


In [13]:
# Train an SVM classifier on the extracted features from the training set
svm_classifier = SVC(kernel='linear', C=1)
svm_classifier.fit(train_features, train_labels)

SVC(C=1, kernel='linear')

In [14]:
# Predict on the test set and evaluate performance
test_predictions = svm_classifier.predict(test_features)
accuracy = accuracy_score(test_labels, test_predictions)
print(f"SVM accuracy on test data: {accuracy * 100:.2f}%")

SVM accuracy on test data: 64.80%


In [15]:
 
print(classification_report(test_labels, test_predictions, target_names=test_generator.class_indices.keys()))


              precision    recall  f1-score   support

   with_mask       0.66      0.56      0.61      1725
without_mask       0.64      0.73      0.68      1818

    accuracy                           0.65      3543
   macro avg       0.65      0.65      0.64      3543
weighted avg       0.65      0.65      0.65      3543

